In [3]:
import requests
import json
from os.path import join, split

import dpath

DANTE_URL = 'https://api.dante.gbv.de'

casserolle = ['amh_causes_role']
real_unit = 'nld_areal_unit'
material = 'kenom_material'

niedersachsen = requests.get(join(DANTE_URL, 'voc', material, 'top'))
niedersachsen.json()

[{'uri': 'http://uri.gbv.de/terminology/kenom_material/4e8b1c94-97f5-44bd-bd5b-0dc3f84e5417',
  'type': ['http://schema.vocnet.org/NonIndexingConcept',
   'http://www.w3.org/2004/02/skos/core#Concept',
   'http://vocab.getty.edu/ontology#GuideTerm'],
  '@context': 'https://gbv.github.io/jskos/context.json',
  'inScheme': [{'uri': 'http://uri.gbv.de/terminology/kenom_material/',
    'prefLabel': {'de': 'KENOM: Material', 'en': 'KENOM: Material'},
    'type': ['http://www.w3.org/2004/02/skos/core#ConceptScheme',
     'http://w3id.org/nkos/nkostype#classification_schema'],
    'notation': ['kenom_material']}],
  'publisher': [{'prefLabel': {'de': 'KENOM, c/o Frank Dührkohp, Digitale Bibliothek, Verbundzentrale des GBV (VZG), Platz der Göttinger Sieben 1, 37073 Göttingen'}}],
  'prefLabel': {'de': 'Metalle', 'en': 'Metals'},
  'narrower': [None]},
 {'uri': 'http://uri.gbv.de/terminology/kenom_material/17f7f3a0-b8ee-4a71-9a39-796605749503',
  'type': ['http://schema.vocnet.org/NonIndexingCo

In [4]:
units_of_niedersachsen = requests.get(join(DANTE_URL, 'descendants'), params = {'uri': niedersachsen.json()[0]['uri'], 'limit': 2000})
units_of_niedersachsen.json()

[{'uri': 'http://uri.gbv.de/terminology/kenom_material/277086d7-bc08-4757-98ba-d6c125276479',
  'type': ['http://www.w3.org/2004/02/skos/core#Concept'],
  '@context': 'https://gbv.github.io/jskos/context.json',
  'inScheme': [{'uri': 'http://uri.gbv.de/terminology/kenom_material/',
    'prefLabel': {'de': 'KENOM: Material', 'en': 'KENOM: Material'},
    'type': ['http://www.w3.org/2004/02/skos/core#ConceptScheme',
     'http://w3id.org/nkos/nkostype#classification_schema'],
    'notation': ['kenom_material']}],
  'publisher': [{'prefLabel': {'de': 'KENOM, c/o Frank Dührkohp, Digitale Bibliothek, Verbundzentrale des GBV (VZG), Platz der Göttinger Sieben 1, 37073 Göttingen'}}],
  'identifier': ['http://nomisma.org/id/ar',
   'https://ikmk.smb.museum/ndp/material/1',
   'http://vocab.getty.edu/aat/300011029',
   'http://d-nb.info/gnd/4133759-1'],
  'notation': ['1'],
  'prefLabel': {'de': 'Silber', 'en': 'Silver'},
  'scopeNote': {'de': ['Für Münzen / Medaillen']},
  'broader': [None],
  

In [3]:

units_of_niedersachsen = units_of_niedersachsen.json()
with open('units_of_niedersachsen.json', 'w') as uon:
    json.dump(units_of_niedersachsen, uon, indent=2)
units_of_niedersachsen[4]

{'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/f672d535-9eea-4dd1-8a2f-52b56b479266',
 'type': ['http://www.w3.org/2004/02/skos/core#Concept',
  'http://d-nb.info/standards/elementset/gnd#PlaceOrGeographicName'],
 '@context': 'https://gbv.github.io/jskos/context.json',
 'inScheme': [{'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/',
   'prefLabel': {'de': 'Niedersächsisches Landesamt für Denkmalpflege: Gebietseinheiten',
    'en': 'Niedersächsisches Landesamt für Denkmalpflege: Gebietseinheiten'},
   'type': ['http://www.w3.org/2004/02/skos/core#ConceptScheme',
    'http://w3id.org/nkos/nkostype#gazetteer'],
   'notation': ['nld_areal_unit'],
   'license': [{'uri': 'http://www.govdata.de/dl-de/by-2-0',
     'notation': ['DL-DE-BY 2.0'],
     'prefLabel': {'de': 'Datenlizenz Deutschland – Namensnennung – 2.0',
      'en': 'Data licence Germany – attribution – 2.0'}}]}],
 'publisher': [{'prefLabel': {'de': 'Landesamt für Geoinformation und Landesvermessung Niedersachsen'}}],

In [16]:
class Node:
    def __init__(self, uri="", prefLabel="", dig=False, dig_deeper=False, *args, **kwargs):
        self.uri = uri
        self.prefLabel = prefLabel
        self.id = split(uri)[-1]
        self.checked_for_children = False
        if dig:
            self.check_for_children(dig_deeper=dig_deeper)
        else:
            self.children = []
            
    def check_for_children(self, dig_deeper=False):
        if not self.checked_for_children:
            descendants = requests.get(join(DANTE_URL, 'descendants'), params = {'uri': self.uri})
            self.children = [Node(dig=dig_deeper, dig_deeper=dig_deeper, **d) for d in descendants.json()]
            self.checked_for_children = True
    
    @property
    def has_children(self):
        if not self.checked_for_children:
            self.check_for_children()
        return bool(len(self.children))
    
    @classmethod
    def from_uri(cls, uri):
        data = requests.get(join(DANTE_URL, 'data'), params={'uri': uri}).json()[0]
        return cls(**data)
    
    def json(self, recursive=False):
        to_return = {'uri': self.uri,
                     'prefLabel': self.prefLabel,
                     'id': self.id,
                     'children': self.children}
        if recursive:
            self.checked_for_children = False
            self.check_for_children(True)
            to_return['children'] = [child.json(True) for child in self.children]
        return to_return
    
    def flatten_children(self, flat_list=[]):
        for child in self.children:
            flat_list.append(child)
            if child.has_children:
                flat_list.extend(child.flatten_children(list(flat_list)))
        return flat_list
    
    def get_idai(self):
        return {self.prefLabel: {'references': [self.uri]}}
        
                


In [5]:
prinzhoefte_json = requests.get(join(DANTE_URL, 'data'), params={'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/97e1dc67-2d57-48ba-b2f7-6aba533d7851'}).json()[0]
prinzhoefte = Node(**prinzhoefte_json)
prinzhoefte.check_for_children(True)
prinzhoefte.has_children

True

In [6]:
prinzhoefte.json(True)

{'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/97e1dc67-2d57-48ba-b2f7-6aba533d7851',
 'prefLabel': {'de': 'Prinzhöfte'},
 'id': '97e1dc67-2d57-48ba-b2f7-6aba533d7851',
 'children': [{'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/9276dc90-628a-431c-ae01-42c75df80ef9',
   'prefLabel': {'zxx': 'Prinzhöfte'},
   'id': '9276dc90-628a-431c-ae01-42c75df80ef9',
   'children': []},
  {'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/a9153ee8-7587-4f8b-b717-002cb18ad2e0',
   'prefLabel': {'zxx': 'Klein Henstedt'},
   'id': 'a9153ee8-7587-4f8b-b717-002cb18ad2e0',
   'children': []},
  {'uri': 'http://uri.gbv.de/terminology/nld_areal_unit/4d99bfdd-6dc0-40d6-9a08-69e700bb8329',
   'prefLabel': {'zxx': 'Horstedt'},
   'id': '4d99bfdd-6dc0-40d6-9a08-69e700bb8329',
   'children': []}]}

In [7]:
prinzhoefte.flatten_children()

In [2]:
DB_CREDENTIALS = ('anonymous', 'xTDLnkJq')
CONFIG_URL = "http://localhost:3001/adox_template/configuration"

conf = requests.get(CONFIG_URL, auth=requests.auth.HTTPBasicAuth(*DB_CREDENTIALS)).json()
conf

{'resource': {'identifier': 'Configuration',
  'id': 'configuration',
  'forms': {'Project': {'hidden': [], 'fields': {}},
   'Operation': {'hidden': [], 'fields': {}},
   'Place': {'hidden': [], 'fields': {}},
   'Trench': {'hidden': [], 'fields': {}},
   'Find': {'hidden': [], 'fields': {}},
   'Feature': {'hidden': [], 'fields': {}},
   'Image': {'hidden': [], 'fields': {}},
   'TypeCatalog': {'hidden': [], 'fields': {}},
   'Type': {'hidden': [], 'fields': {}},
   'Profile': {'fields': {}, 'hidden': []},
   'Befundkomplex': {'color': '#c45010',
    'parent': 'Feature',
    'fields': {},
    'groups': [{'name': 'stem',
      'fields': ['identifier', 'category', 'shortDescription', 'isSameAs']},
     {'name': 'position',
      'fields': ['geometry',
       'isPresentIn',
       'borders',
       'cuts',
       'isCutBy',
       'isAbove',
       'isBelow']},
     {'name': 'time',
      'fields': ['period',
       'dating',
       'isAfter',
       'isBefore',
       'isContemporaryWi

In [3]:
import dpath.util as dpu
value_lists = dpu.search(conf, "**/valuelists/*")
value_lists

{'resource': {'valuelists': {'adox_template:Vollwert': {'values': {'Voll': {'label': {'de': 'Voll',
       'en': 'Full'},
      'description': {'de': 'Die Fülle', 'en': 'The fullness'}},
     'Wert': {'references': ['https://trappotte.party'],
      'label': {'de': 'Wert', 'en': 'Value'},
      'description': {'de': 'Was das dann wert ist',
       'en': "What it's all about"}}},
    'createdBy': 'anonymous',
    'creationDate': '2022-05-19',
    'references': ['https://schlimme-gegend.de'],
    'description': {'de': 'Eine vollwertige Werteliste',
     'en': 'A list full of values'}}}}}

In [4]:
print(json.dumps(value_lists, indent=2))

{
  "resource": {
    "valuelists": {
      "adox_template:Vollwert": {
        "values": {
          "Voll": {
            "label": {
              "de": "Voll",
              "en": "Full"
            },
            "description": {
              "de": "Die F\u00fclle",
              "en": "The fullness"
            }
          },
          "Wert": {
            "references": [
              "https://trappotte.party"
            ],
            "label": {
              "de": "Wert",
              "en": "Value"
            },
            "description": {
              "de": "Was das dann wert ist",
              "en": "What it's all about"
            }
          }
        },
        "createdBy": "anonymous",
        "creationDate": "2022-05-19",
        "references": [
          "https://schlimme-gegend.de"
        ],
        "description": {
          "de": "Eine vollwertige Werteliste",
          "en": "A list full of values"
        }
      }
    }
  }
}
